In [ ]:
!pip install torch
!pip install dgl==0.9.0
!pip install torch_geometric
!pip install tensorflow
!pip install transformers

In [ ]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
import os
from natsort import natsorted
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import remove_stopwords
from gensim.parsing.preprocessing import STOPWORDS
from nltk.stem import WordNetLemmatizer
from gensim.models import Word2Vec
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import GridSearchCV
import string
from nltk.tokenize import word_tokenize
from nltk.util import ngrams
import numpy as np
from nltk.corpus import stopwords
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt
from gensim.models import FastText
import matplotlib.pyplot as plt
import os

os.environ["DGLBACKEND"] = "pytorch"
import dgl
import dgl.data
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch_geometric
import numpy as np
import random
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import numpy as np
from sklearn.metrics import mean_squared_error
from keras.models import Sequential
from keras.layers import LSTM, Dropout, Dense,Bidirectional
from tensorflow.keras.regularizers import l2
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch_geometric
import numpy as np
from numpy.linalg import norm
from dgl.nn import GraphConv
from IPython.display import FileLink

import os
import zipfile
import numpy as np
from nltk.tokenize import word_tokenize
from natsort import natsorted
from nltk.corpus import stopwords
from transformers import BertTokenizer, BertModel
import torch
import pandas as pd

/usr/local/lib/python3.10/dist-packages/dgl/backend/pytorch/sparse.py:104: FutureWarning: `torch.cuda.amp.custom_fwd(args...)` is deprecated. Please use `torch.amp.custom_fwd(args..., device_type='cuda')` instead.
  @custom_fwd(cast_inputs=th.float16)
/usr/local/lib/python3.10/dist-packages/dgl/backend/pytorch/sparse.py:128: FutureWarning: `torch.cuda.amp.custom_bwd(args...)` is deprecated. Please use `torch.amp.custom_bwd(args..., device_type='cuda')` instead.
  def backward(ctx, dZ):
/usr/local/lib/python3.10/dist-packages/dgl/backend/pytorch/sparse.py:177: FutureWarning: `torch.cuda.amp.custom_fwd(args...)` is deprecated. Please use `torch.amp.custom_fwd(args..., device_type='cuda')` instead.
  @custom_fwd(cast_inputs=th.float16)
/usr/local/lib/python3.10/dist-packages/dgl/backend/pytorch/sparse.py:207: FutureWarning: `torch.cuda.amp.custom_bwd(args...)` is deprecated. Please use `torch.amp.custom_bwd(args..., device_type='cuda')` instead.
  def backward(ctx, *dZ):
/usr/local/lib/py

In [ ]:
class MySentences:
    def __init__(self, zip_file_path, extract_to):
        self.zip_file_path = zip_file_path
        self.extract_to = extract_to
        self.root_directory = self.unzip_and_get_root_directory()

    def unzip_and_get_root_directory(self):
        with zipfile.ZipFile(self.zip_file_path, 'r') as zip_ref:
            zip_ref.extractall(self.extract_to)
        return os.path.join(self.extract_to, 'Image_data')  # Adjust if folder name is different

    def __iter__(self):
        file_list = [file for file in os.listdir(self.root_directory) if not file.startswith('.DS_Store')]
        file_list = natsorted(file_list)

        for subdirectory in file_list:
            subdirectory_path = os.path.join(self.root_directory, subdirectory, 'txt')
            if os.path.exists(subdirectory_path):
                file_list_t = [file for file in os.listdir(subdirectory_path) if not file.startswith('.DS_Store')]
                file_list_t = natsorted(file_list_t)

                for filename in file_list_t:
                    file_path = os.path.join(subdirectory_path, filename)
                    with open(file_path, 'r') as file:
                        A = []
                        for line in file:
                            line = line.lower()
                            line = pun_remove(line)
                            line = word_tokenize(line)
                            line_Without_stopwords = [word for word in line if word not in stopwords_list]
                            if line_Without_stopwords:
                                A.append(line_Without_stopwords)
                        A = list(np.concatenate(A))
                        yield ' '.join(A)  # Combine tokens into a sentence


In [ ]:
# # Initialize BERT model and tokenizer
# # tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
# # model = BertModel.from_pretrained('bert-base-uncased')

# from transformers import DistilBertTokenizer, DistilBertModel
# tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')
# model = DistilBertModel.from_pretrained('distilbert-base-uncased')

# # Define function to get sentence embeddings from BERT
# def get_bert_embedding(sentence):
#     inputs = tokenizer(sentence, return_tensors='pt', padding=True, truncation=True)
#     with torch.no_grad():
#         outputs = model(**inputs)
#     # Get the embeddings for the [CLS] token (sentence-level representation)
#     cls_embedding = outputs.last_hidden_state[:, 0, :]
#     return cls_embedding.squeeze().numpy()

from transformers import RobertaTokenizer, RobertaModel
import torch

# Initialize RoBERTa model and tokenizer
tokenizer = RobertaTokenizer.from_pretrained('roberta-base')
model = RobertaModel.from_pretrained('roberta-base')

# Define function to get sentence embeddings from RoBERTa
def get_roberta_embedding(sentence):
    inputs = tokenizer(sentence, return_tensors='pt', padding=True, truncation=True)
    with torch.no_grad():
        outputs = model(**inputs)
    # Get the embeddings for the [CLS]-equivalent token ([<s>] in RoBERTa)
    cls_embedding = outputs.last_hidden_state[:, 0, :]  # [<s>] is at position 0 in RoBERTa
    return cls_embedding.squeeze().numpy()

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-base and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
# Usage
zip_file_path = '/Image_data.zip'
extract_to = '/Extracted_image_data'

# Create an instance of MySentences with the zip file path
sentences = MySentences(zip_file_path, extract_to)

# Iterate over the sentences, extract BERT embeddings, and collect them in a DataFrame
embeddings = []
for sentence in sentences:
    embedding = get_roberta_embedding(sentence)
    embeddings.append(embedding)

# Create a DataFrame with BERT embeddings
X_train_df = pd.DataFrame(embeddings)

# Save the DataFrame to a CSV file
X_train_df.to_csv('Text_features_roberta.csv', index=False)

# Display download link for the CSV file
from IPython.display import FileLink
FileLink('Text_features_roberta.csv')

/content/Text_features_roberta.csv

In [ ]:
from google.colab import files
files.download('Text_features_roberta.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
import nltk
nltk.download('stopwords')
nltk.download('punkt_tab')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

In [ ]:
stopwords_list = stopwords.words('english')
stopwords_list.append('33')
stopwords_list.append('52')
stopwords_list


remove=string.punctuation
new_symbol = '’'
new_symbol1 = '‘'
new_symbol2 = '–'
new_symbol3 = '—'

remove = remove + new_symbol
remove = remove + new_symbol1
remove = remove + new_symbol2
remove = remove + new_symbol3



def pun_remove(data):
    for char in remove:
        data=data.replace(char,' ')
    data=data.replace('“','')
    data=data.replace('”','')
    return data

In [ ]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [ ]:
# Preparing Node Data
Nodes_Data = pd.DataFrame({'Id': range(len(embeddings)), 'features': embeddings})
labels = pd.read_csv("/AnimalLabels.csv")
labels['majority_vote'] = labels.mode(axis=1, numeric_only=True).astype(int)
Nodes_Data['label'] = labels['majority_vote']

# Generating Edge Data for pairwise nodes with cosine similarity
Src_ID, Dst_ID, Src_feature, Dst_feature, edge_weight = [], [], [], [], []
for i in range(len(embeddings)):
    for j in range(i + 1, len(embeddings)):
        Src_ID.append(i)
        Dst_ID.append(j)
        src_feat, dst_feat = np.array(embeddings[i]), np.array(embeddings[j])
        similarity = np.dot(src_feat, dst_feat) / (norm(src_feat) * norm(dst_feat))
        Src_feature.append(src_feat)
        Dst_feature.append(dst_feat)
        edge_weight.append(max(similarity, 0))  # Ensure non-negative weights

Edge_Data = pd.DataFrame({
    'Src Id': Src_ID,
    'Dst Id': Dst_ID,
    'Src_feature': Src_feature,
    'Dst_feature': Dst_feature,
    'edge weights': edge_weight
})

# Save edge data to CSV
Edge_Data.to_csv('Edge_Data.csv', index=False)

# Provide download links
from IPython.display import FileLink
FileLink('Text_features_bert.csv'), FileLink('Edge_Data.csv')

(/content/Text_features_bert.csv, /content/Edge_Data.csv)

In [ ]:
print(Edge_Data['edge weights'].describe())
ninetyfive_percentile = Edge_Data['edge weights'].quantile(0.95)
print("95th percentile of edge weights column:", ninetyfive_percentile)

count    5565.000000
mean        0.996912
std         0.000929
min         0.990042
25%         0.996473
50%         0.997047
75%         0.997511
max         1.000000
Name: edge weights, dtype: float64
95th percentile of edge weights column: 0.9980849146842956


In [ ]:
percentiles=[round(0.996473,4),round(0.997047,4),round(0.997511,4),round(0.9980849146842956,4)]

In [ ]:
import os
os.environ["DGLBACKEND"] = "pytorch"

import dgl
import torch
import numpy as np
from dgl.data import DGLDataset

# Assuming `Nodes_Data` and `Edge_Data` DataFrames are already defined

class CustomGraphDataset(DGLDataset):
    def __init__(self, threshold):
        self.threshold = threshold
        super().__init__(name="custom_graph")

    def process(self):
        # Define the list to store indices of edges to be removed
        edge_remove = []

        # Extract edge weights and apply threshold filtering
        edge_weights = Edge_Data["edge weights"].to_numpy()
        for i in range(len(edge_weights)):
            if edge_weights[i] <= self.threshold:
                edge_remove.append(i)
            else:
                edge_weight.append(edge_weights[i])

        # Convert node features and labels
        features_array = np.array(Nodes_Data["features"].tolist(), dtype=float)
        node_features = torch.from_numpy(features_array).float()
        node_labels = torch.tensor(Nodes_Data["label"].values, dtype=torch.long)

        # Convert edge features and source/destination IDs
        edge_features = torch.tensor(edge_weights, dtype=torch.float32)
        edges_src = torch.tensor(Edge_Data["Src Id"].values, dtype=torch.long)
        edges_dst = torch.tensor(Edge_Data["Dst Id"].values, dtype=torch.long)

        # Create the DGL graph
        self.graph = dgl.graph((edges_src, edges_dst), num_nodes=Nodes_Data.shape[0])
        self.graph.ndata["feat"] = node_features
        self.graph.ndata["label"] = node_labels
        self.graph.edata["weight"] = edge_features

        # Remove edges below threshold
        self.graph = dgl.remove_edges(self.graph, torch.tensor(edge_remove, dtype=torch.long))

        # Create train, validation, and test masks
        n_nodes = Nodes_Data.shape[0]
        n_train = int(n_nodes * 0.6)
        n_val = int(n_nodes * 0.2)
        train_mask = torch.zeros(n_nodes, dtype=torch.bool)
        val_mask = torch.zeros(n_nodes, dtype=torch.bool)
        test_mask = torch.zeros(n_nodes, dtype=torch.bool)
        train_mask[:n_train] = True
        val_mask[n_train:n_train + n_val] = True
        test_mask[n_train + n_val:] = True

        self.graph.ndata["train_mask"] = train_mask
        self.graph.ndata["val_mask"] = val_mask
        self.graph.ndata["test_mask"] = test_mask

    def __getitem__(self, idx):
        return self.graph

    def __len__(self):
        return 1

# Set threshold value
threshold = 0.9275
dataset = CustomGraphDataset(threshold)
graph = dataset[0]

# Print the graph to confirm its structure
print(graph)


Graph(num_nodes=106, num_edges=5565,
      ndata_schemes={'feat': Scheme(shape=(768,), dtype=torch.float32), 'label': Scheme(shape=(), dtype=torch.int64), 'train_mask': Scheme(shape=(), dtype=torch.bool), 'val_mask': Scheme(shape=(), dtype=torch.bool), 'test_mask': Scheme(shape=(), dtype=torch.bool)}
      edata_schemes={'weight': Scheme(shape=(), dtype=torch.float32)})


In [ ]:
# Verify column names in Edge_Data
print("Edge_Data columns:", Edge_Data.columns)

# Adjust column names here if necessary
src_col = 'Src Id'  # Replace with the actual name for source column if different
dst_col = 'Dst Id'  # Replace with the actual name for destination column if different
edge_weight_col = 'edge weights'  # Replace with the actual name for edge weight column if different

# Updated Code with Column Names Adjusted
new_g = dgl.compact_graphs(graph)
train_mask = graph.ndata["train_mask"]

# Set train_mask to True for the first 106 nodes
for i in range(106):
    train_mask[i] = True

# Set specified indices to False in train_mask
indices_to_change = [4, 105, 84, 27, 98, 88, 18, 65, 9, 2, 5, 49, 99, 69, 86, 67, 7, 28, 78, 70, 18, 74]
train_mask[indices_to_change] = False
graph.ndata["train_mask"] = train_mask

# Set test_mask as the inverse of train_mask
test_mask = ~train_mask
graph.ndata["test_mask"] = test_mask

# Filter Edge_Data for training edges (exclude edges connecting nodes in indices_to_change)
Edge_Data_train = Edge_Data[
    ~Edge_Data[src_col].isin(indices_to_change) &
    ~Edge_Data[dst_col].isin(indices_to_change) &
    (Edge_Data[edge_weight_col] > 0.8258)
]

# Filter Edge_Data for testing edges (include only edges connecting nodes in indices_to_change)
Edge_Data_test = Edge_Data[
    Edge_Data[src_col].isin(indices_to_change) &
    Edge_Data[dst_col].isin(indices_to_change) &
    (Edge_Data[edge_weight_col] > 0.8258)
]

# Create and save the dense adjacency matrix for the graph
adj_matrix = graph.adj(etype=None)
dense_matrix = adj_matrix.to_dense().numpy()

# Add node index as a new column to dense_matrix
new_column = np.arange(0, dense_matrix.shape[0]).reshape((106, 1))
dense_matrix = np.concatenate((new_column, dense_matrix), axis=1)

# Add row index as a new row to the top of dense_matrix
new_row = np.insert(np.arange(0, 106), 0, 0).reshape((1, 107))
result_matrix = np.vstack([new_row, dense_matrix])

# Save dense matrix to CSV
np.savetxt('dense_matrix_avg.csv', result_matrix, delimiter=',', fmt='%d')
FileLink(r'dense_matrix_avg.csv')

# Generate subgraph for training set and save its adjacency matrix
sg_train = dgl.node_subgraph(graph, train_mask)
sg_adjacency_train = sg_train.adj()
dense_matrix_sg_train = sg_adjacency_train.to_dense().numpy()

np.savetxt('dense_matrix_sg_train.csv', dense_matrix_sg_train, delimiter=',', fmt='%d')
FileLink(r'dense_matrix_sg_train.csv')

# Generate subgraph for test set and save its adjacency matrix
sg_test = dgl.node_subgraph(graph, test_mask)
sg_adjacency_test = sg_test.adj()
dense_matrix_sg_test = sg_adjacency_test.to_dense().numpy()

np.savetxt('dense_matrix_sg_test.csv', dense_matrix_sg_test, delimiter=',', fmt='%d')
FileLink(r'dense_matrix_sg_test.csv')


Edge_Data columns: Index(['Src Id', 'Dst Id', 'Src_feature', 'Dst_feature', 'edge weights'], dtype='object')


/content/dense_matrix_sg_test.csv

In [ ]:
seed = 42
torch.manual_seed(seed)
random.seed(seed)
np.random.seed(seed)
# class GCN(nn.Module):
#     def __init__(self, in_feats, h_feats, num_classes):
#         super(GCN, self).__init__()
#         self.conv1 = GraphConv(in_feats, h_feats,norm='both')
#         self.conv2 = GraphConv(h_feats, num_classes)

#     def forward(self, g, in_feat):
#         in_feat = in_feat.float()
#         h = self.conv1(graph, in_feat)
#         h = F.leaky_relu(h)
#         h = self.conv2(graph, h)
#         return h

# Define the GCN model
class GCN(nn.Module):
    def __init__(self, in_feats, h_feats, num_classes):
        super(GCN, self).__init__()
        self.conv1 = GraphConv(in_feats, h_feats, norm='both')
        self.conv2 = GraphConv(h_feats, num_classes)

    def forward(self, g, in_feat):
        in_feat = in_feat.float()
        h = self.conv1(g, in_feat)
        h = F.leaky_relu(h)
        h = self.conv2(g, h)
        return h

In [ ]:
los=[]
train_accuracy=[]
test_accuracy=[]
epoch=[]

sg_train = dgl.add_self_loop(sg_train)
sg_test = dgl.add_self_loop(sg_test)

def train(sg_train, sg_test, model):
    model.train()
    optimizer = torch.optim.Adam(model.parameters(), lr=1)
    best_train_acc = 0
    best_test_acc = 0

    features_train = sg_train.ndata["feat"]
    labels = graph.ndata["label"]
    train_mask = graph.ndata["train_mask"]
    test_mask = graph.ndata["test_mask"]

    for e in range(1000):
        # Forward pass
        logits = model(sg_train, features_train)
        pred = logits.argmax(1)

        labels = graph.ndata["label"].long()
        loss = F.cross_entropy(logits, labels[train_mask])

        # Test set predictions
        with torch.no_grad():
            model.eval()
            logits_test = model(sg_test, sg_test.ndata["feat"])
            pred_test = logits_test.argmax(1)

        # Calculate accuracies
        train_acc = (pred == labels[train_mask]).float().mean()
        test_acc = (pred_test == labels[test_mask]).float().mean()

        # Track best train and test accuracy
        best_train_acc = max(best_train_acc, train_acc)
        best_test_acc = max(best_test_acc, test_acc)

        # Backward pass
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # Log metrics every 5 epochs
        if e % 5 == 0:
            epoch.append(e)
            los.append(round(loss.item(), 3))
            train_accuracy.append(round(train_acc.item(), 3))
            test_accuracy.append(round(test_acc.item(), 3))
            print(
                f"In epoch {e}, loss: {loss:.3f}, train acc: {train_acc:.3f} (train {best_train_acc:.3f}), test acc: {test_acc:.3f} (best {best_test_acc:.3f})"
            )

# Initialize and train the model
model = GCN(graph.ndata["feat"].shape[1], 22, 2)
train(sg_train, sg_test, model)

/usr/local/lib/python3.10/dist-packages/dgl/backend/pytorch/tensor.py:352: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  assert input.numel() == input.storage().size(), "Cannot convert view " \


In epoch 0, loss: 0.655, train acc: 0.647 (train 0.647), test acc: 0.667 (best 0.667)
In epoch 5, loss: 6.654, train acc: 0.365 (train 0.647), test acc: 0.429 (best 0.667)
In epoch 10, loss: 6.925, train acc: 0.647 (train 0.647), test acc: 0.667 (best 0.667)
In epoch 15, loss: 0.883, train acc: 0.529 (train 0.647), test acc: 0.619 (best 0.667)
In epoch 20, loss: 1.705, train acc: 0.647 (train 0.647), test acc: 0.667 (best 0.667)
In epoch 25, loss: 1.452, train acc: 0.647 (train 0.647), test acc: 0.667 (best 0.667)
In epoch 30, loss: 1.247, train acc: 0.647 (train 0.647), test acc: 0.667 (best 0.667)
In epoch 35, loss: 0.683, train acc: 0.600 (train 0.659), test acc: 0.571 (best 0.667)
In epoch 40, loss: 0.664, train acc: 0.647 (train 0.659), test acc: 0.667 (best 0.667)
In epoch 45, loss: 0.770, train acc: 0.647 (train 0.659), test acc: 0.667 (best 0.667)
In epoch 50, loss: 0.648, train acc: 0.647 (train 0.659), test acc: 0.667 (best 0.667)
In epoch 55, loss: 0.661, train acc: 0.647 (t

In [ ]:
import os

os.environ["DGLBACKEND"] = "pytorch"
import dgl
import torch
from dgl.data import DGLDataset


class KarateClubDataset(DGLDataset):
    def __init__(self,params):
        self.params=params
        #self.threshold=threshold
        super().__init__(name="karate_club")
        self.process()
    def process(self):
        edge_remove=[]
        C=edge_weights
        for i in range(0,len(C)):
            if C[i]<=self.params['percentile']:
                edge_remove.append(i)
        nodes_data = Nodes_Data
        edges_data = Edge_Data
        features_array = np.array(nodes_data["features"].tolist(), dtype=float)
        node_features = torch.from_numpy(features_array)
        node_labels = torch.from_numpy(
                      nodes_data["label"].astype("category").cat.codes.to_numpy()
                       ).clone().detach()
        edge_features = torch.from_numpy(edges_data["edge weights"].to_numpy())
        edges_src = torch.from_numpy(edges_data["Src Ids"].to_numpy())
        edges_dst = torch.from_numpy(edges_data["Dst Ids"].to_numpy())
        self.graph = dgl.graph(
            (edges_src, edges_dst), num_nodes=nodes_data.shape[0]
        )
        self.graph.ndata["feat"] = node_features
        self.graph.ndata["label"] = node_labels
        self.graph.edata["weight"] = edge_features

        self.graph=dgl.remove_edges(self.graph, torch.tensor(edge_remove))
        n_nodes = nodes_data.shape[0]
        n_train = int(n_nodes * 0.6)
        n_val = int(n_nodes * 0.2)
        train_mask = torch.zeros(n_nodes, dtype=torch.bool)
        val_mask = torch.zeros(n_nodes, dtype=torch.bool)
        test_mask = torch.zeros(n_nodes, dtype=torch.bool)
        train_mask[:n_train] = True
        val_mask[n_train : n_train + n_val] = True
        test_mask[n_train + n_val :] = True
        self.graph.ndata["train_mask"] = train_mask
        self.graph.ndata["val_mask"] = val_mask
        self.graph.ndata["test_mask"] = test_mask

    def __getitem__(self, i):
        return self.graph

    def __len__(self):
        return 1

In [ ]:
import os
import dgl
import torch
import numpy as np
from dgl.data import DGLDataset

os.environ["DGLBACKEND"] = "pytorch"

class KarateClubDataset(DGLDataset):
    def __init__(self, params):
        self.params = params
        super().__init__(name="karate_club")
        self.process()

    def process(self):
        # Edge removal based on the percentile value
        edge_remove = []
        C = edge_weights  # Assuming `edge_weights` is predefined elsewhere
        for i in range(len(C)):
            if C[i] <= self.params['percentile']:
                edge_remove.append(i)

        nodes_data = Nodes_Data  # Assuming `Nodes_Data` is a predefined dataframe
        edges_data = Edge_Data  # Assuming `Edge_Data` is a predefined dataframe

        # Prepare node features, labels, and edge features
        features_array = np.array(nodes_data["features"].tolist(), dtype=float)
        node_features = torch.from_numpy(features_array)
        node_labels = torch.from_numpy(
            nodes_data["label"].astype("category").cat.codes.to_numpy()
        ).clone().detach()

        edge_features = torch.from_numpy(edges_data["edge weights"].to_numpy())
        edges_src = torch.from_numpy(edges_data["Src Ids"].to_numpy())
        edges_dst = torch.from_numpy(edges_data["Dst Ids"].to_numpy())

        # Create the graph
        self.graph = dgl.graph(
            (edges_src, edges_dst), num_nodes=nodes_data.shape[0]
        )
        self.graph.ndata["feat"] = node_features
        self.graph.ndata["label"] = node_labels
        self.graph.edata["weight"] = edge_features

        # Remove edges based on edge removal condition
        self.graph = dgl.remove_edges(self.graph, torch.tensor(edge_remove))

        # Add self-loops to prevent 0-in-degree error
        self.graph = dgl.add_self_loop(self.graph)

        # Splitting into train, validation, and test masks
        n_nodes = nodes_data.shape[0]
        n_train = int(n_nodes * 0.6)
        n_val = int(n_nodes * 0.2)

        train_mask = torch.zeros(n_nodes, dtype=torch.bool)
        val_mask = torch.zeros(n_nodes, dtype=torch.bool)
        test_mask = torch.zeros(n_nodes, dtype=torch.bool)

        train_mask[:n_train] = True
        val_mask[n_train:n_train + n_val] = True
        test_mask[n_train + n_val:] = True

        self.graph.ndata["train_mask"] = train_mask
        self.graph.ndata["val_mask"] = val_mask
        self.graph.ndata["test_mask"] = test_mask

    def __getitem__(self, i):
        return self.graph

    def __len__(self):
        return 1


In [ ]:
# seed = 42
# torch.manual_seed(seed)
# random.seed(seed)
# np.random.seed(seed)


# class GCN(nn.Module):
#     def __init__(self,in_feats,params,num_classes):
#         super(GCN, self).__init__()
#         self.conv1 = GraphConv(in_feats, params['output_features'])
#         self.conv2 = GraphConv(params['output_features'], num_classes)

#     def forward(self, g,in_feat,params):
#         in_feat = in_feat.float()
#         h = self.conv1(g, in_feat)
#         h = getattr(F, params['activation'])(h)
#         h = self.conv2(g, h)
#         return h

class GCN(nn.Module):
    def __init__(self, in_feats, params, num_classes):
        super(GCN, self).__init__()
        # Initialize the layers based on provided params
        self.conv1 = GraphConv(in_feats, params['output_features'])
        self.conv2 = GraphConv(params['output_features'], num_classes)

    def forward(self, g, in_feat, params):
        # Ensure features are in float format
        in_feat = in_feat.float()

        # Perform first graph convolution
        h = self.conv1(g, in_feat)

        # Apply activation function based on `params`
        h = getattr(F, params['activation'])(h)  # Dynamic activation based on params

        # Perform second graph convolution
        h = self.conv2(g, h)
        return h

In [ ]:
# los=[]
# train_accuracy=[]
# test_accuracy=[]
# TT=[]
# CT=[]
# epoch=[]
# seed = 42
# torch.manual_seed(seed)
# random.seed(seed)
# np.random.seed(seed)
# import torch.optim as optim
# import torch.nn.functional as F
# def train(g,sg_train,sg_test, model,params):
#     model.train()
#     optimizer = getattr(optim, params['optimizer'])(model.parameters(), lr= params['learning_rate'])
#     best_train_acc = 0
#     best_test_acc = 0
#     corresponding_test=0

#     features_train = sg_train.ndata["feat"]
#     labels = g.ndata["label"]
#     train_mask = g.ndata["train_mask"]
#     test_mask = g.ndata["test_mask"]


#     for e in range(100):
#         # Forward
#         logits = model(sg_train,features_train,params)


#         # Compute prediction
#         pred = logits.argmax(1)


#         labels = g.ndata["label"].long()
#         if params['loss'] == 'binary_cross_entropy':
#             loss = getattr(F, params['loss'])(torch.sigmoid(logits), labels[train_mask])
#         else:
#             loss = getattr(F, params['loss'])(logits, labels[train_mask])

#         with torch.no_grad():
#             model.eval()
#             logits_test = model(sg_test,sg_test.ndata["feat"],params)


#         pred_test = logits_test.argmax(1)

#         train_acc = (pred == labels[train_mask]).float().mean()
#         test_acc = (pred_test == labels[test_mask]).float().mean()

#         if best_train_acc < train_acc:
#             best_train_acc = train_acc
#             corresponding_test=test_acc
#         if best_test_acc < test_acc:
#             best_test_acc = test_acc


#         # Backward
#         optimizer.zero_grad()
#         loss.backward()
#         optimizer.step()


#         if e % 5 == 0:
#             epoch.append(e)
#             los.append(round(loss.item(),3))
#             train_accuracy.append(round(train_acc.item(),3))

#     TT.append({'train_accuracy':best_train_acc})
#     CT.append({'test_accuracy': corresponding_test})

#     print(f'Train Accuracy: {best_train_acc}, Test Accuracy: {corresponding_test}')
#     return best_train_acc

import torch.optim as optim
import torch.nn.functional as F

# Lists to store metrics for later use
los = []
train_accuracy = []
test_accuracy = []
TT = []
CT = []
epoch = []

# Set random seeds for reproducibility
seed = 42
torch.manual_seed(seed)
random.seed(seed)
np.random.seed(seed)

def train(g, sg_train, sg_test, model, params):
    model.train()

    # Optimizer setup
    optimizer = getattr(optim, params['optimizer'])(model.parameters(), lr=params['learning_rate'])

    # Initialize tracking variables for best accuracy
    best_train_acc = 0
    best_test_acc = 0
    corresponding_test = 0

    # Retrieve the relevant graph and node data
    features_train = sg_train.ndata["feat"]
    labels = g.ndata["label"]
    train_mask = g.ndata["train_mask"]
    test_mask = g.ndata["test_mask"]

    # Training loop
    for e in range(100):
        # Forward pass
        logits = model(sg_train, features_train, params)

        # Compute prediction
        pred = logits.argmax(1)

        # Loss calculation
        if params['loss'] == 'binary_cross_entropy':
            loss = getattr(F, params['loss'])(torch.sigmoid(logits), labels[train_mask])
        else:
            loss = getattr(F, params['loss'])(logits, labels[train_mask])

        # Evaluate performance on the test graph
        with torch.no_grad():
            model.eval()
            logits_test = model(sg_test, sg_test.ndata["feat"], params)

        pred_test = logits_test.argmax(1)

        # Calculate training and testing accuracies
        train_acc = (pred == labels[train_mask]).float().mean()
        test_acc = (pred_test == labels[test_mask]).float().mean()

        # Track the best accuracies
        if best_train_acc < train_acc:
            best_train_acc = train_acc
            corresponding_test = test_acc
        if best_test_acc < test_acc:
            best_test_acc = test_acc

        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # Logging every 5 epochs
        if e % 5 == 0:
            epoch.append(e)
            los.append(round(loss.item(), 3))
            train_accuracy.append(round(train_acc.item(), 3))

    # Store results for later
    TT.append({'train_accuracy': best_train_acc})
    CT.append({'test_accuracy': corresponding_test})

    # Print final accuracy results
    print(f'Train Accuracy: {best_train_acc}, Test Accuracy: {corresponding_test}')

    return best_train_acc


In [ ]:
!pip install optuna
import optuna

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 364.4/364.4 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.5/233.5 kB 13.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 4.3 MB/s eta 0:00:00


In [ ]:
# def objective(trial):
#     percentile_values=[percentiles[0],percentiles[1],percentiles[2],percentiles[3]]
#     learning_rate=[0.01,0.1,1,10,100,200]
#     output_features=[10,12,14,16,18,20,22]
#     params={'output_features':trial.suggest_categorical('output_features',output_features),
#            'optimizer':trial.suggest_categorical('optimizer',["Adam", "RMSprop", "SGD"]),
#            'percentile':trial.suggest_categorical('percentile',percentile_values),
#            'learning_rate':trial.suggest_categorical('learning_rate',learning_rate),
#            'loss':trial.suggest_categorical('loss',['cross_entropy']),
#            'activation':trial.suggest_categorical('activation',['relu','selu','elu','leaky_relu','tanh'])}
#     dataset = KarateClubDataset(params)
#     g = dataset[0]
#     train_mask = g.ndata["train_mask"]
#     for i in range(106):
#         train_mask[i] = True

#     indices_to_change = [4, 105, 84, 27, 98, 88, 18, 65, 9, 2, 5, 49, 99, 69, 86, 67, 7, 28, 78, 70, 18, 74]
#     train_mask[indices_to_change] = False
#     g.ndata["train_mask"]=train_mask
#     test_mask = ~train_mask
#     g.ndata["test_mask"]=test_mask
#     sg_train=dgl.node_subgraph(g, train_mask)
#     sg_test = dgl.node_subgraph(g, test_mask)
#     model=GCN(g.ndata["feat"].shape[1],params, 2)
#     accuracy=train(g,sg_train,sg_test,model,params)
#     return accuracy

import torch
import optuna
import dgl
from dgl.data import KarateClubDataset

def objective(trial):
    # Define the possible values for each hyperparameter
    percentile_values = [percentiles[0], percentiles[1], percentiles[2], percentiles[3]]
    learning_rate = [0.0001,0.001,0.01,0.1]
    output_features = [10, 12, 14, 16, 18, 20, 22]

    # # Define the params dictionary with values for hyperparameter optimization
    # params = {
    #     'output_features': trial.suggest_categorical('output_features', output_features),
    #     'optimizer': trial.suggest_categorical('optimizer', ["Adam", "RMSprop", "SGD"]),
    #     'percentile': trial.suggest_categorical('percentile', percentile_values),
    #     'learning_rate': trial.suggest_categorical('learning_rate', learning_rate),
    #     'loss': trial.suggest_categorical('loss', ['cross_entropy']),
    #     'activation': trial.suggest_categorical('activation', ['relu', 'selu', 'elu', 'leaky_relu', 'tanh'])
    # }

    params = {
    'output_features': trial.suggest_categorical('output_features', output_features),
    'optimizer': trial.suggest_categorical('optimizer', ["Adam", "RMSprop", "SGD"]),
    'percentile': trial.suggest_categorical('percentile', percentile_values),
    'learning_rate': trial.suggest_float('learning_rate', 1e-5, 1e-1, log=True),
    'loss': trial.suggest_categorical('loss', ['cross_entropy']),
    'activation': trial.suggest_categorical('activation', ['relu', 'selu', 'elu']),
    'dropout': trial.suggest_float('dropout', 0.2, 0.5),  # Adding dropout hyperparameter
}

    # Load dataset and setup graph
    dataset = KarateClubDataset()
    g = dataset[0]

    # Initialize node features as random values (if not already present in the dataset)
    if "feat" not in g.ndata:
        g.ndata["feat"] = torch.randn(g.number_of_nodes(), 10)  # Here, we use 10 as the feature dimension, but adjust as necessary

    # Add self-loop to avoid zero-in-degree nodes
    g = dgl.add_self_loop(g)

    # Initialize train_mask and test_mask
    g.ndata["train_mask"] = torch.zeros(g.number_of_nodes(), dtype=torch.bool)
    g.ndata["test_mask"] = torch.zeros(g.number_of_nodes(), dtype=torch.bool)

    # Modify train_mask for training set (set to True for the available nodes, up to g.number_of_nodes())
    train_mask = g.ndata["train_mask"]
    for i in range(g.number_of_nodes()):  # Use number of nodes instead of fixed 106
        train_mask[i] = True

    # Manually set indices to False for testing set (ensure indices are within bounds)
    indices_to_change = [4, 105, 84, 27, 98, 88, 18, 65, 9, 2, 5, 49, 99, 69, 86, 67, 7, 28, 78, 70, 18, 74]
    # Filter out indices that exceed the available number of nodes
    indices_to_change = [i for i in indices_to_change if i < g.number_of_nodes()]
    train_mask[indices_to_change] = False
    g.ndata["train_mask"] = train_mask
    test_mask = ~train_mask
    g.ndata["test_mask"] = test_mask

    # Create subgraphs for training and testing
    sg_train = dgl.node_subgraph(g, train_mask)
    sg_test = dgl.node_subgraph(g, test_mask)

    # Initialize GCN model with parameters from the trial
    model = GCN(g.ndata["feat"].shape[1], params, 2)

    # Train the model and get the best accuracy
    accuracy = train(g, sg_train, sg_test, model, params)

    return accuracy


In [24]:
# study = optuna.create_study(direction="maximize")
# study.optimize(objective, n_trials=2520)
import time

# Initialize the study
study = optuna.create_study(direction="maximize")

# Measure the total time for all trials
start_time = time.time()  # Start time before optimization

# Perform the optimization
study.optimize(objective, n_trials=2520)

end_time = time.time()  # End time after optimization

# Calculate total compute time
total_compute_time = end_time - start_time

print(f"Total computation time for all 2520 trials: {total_compute_time:.4f} seconds")

[I 2024-11-20 22:46:13,251] A new study created in memory with name: no-name-814f0895-c8bb-4eb1-85f7-e50bc7eea31e
[I 2024-11-20 22:46:14,419] Trial 0 finished with value: 0.3076923191547394 and parameters: {'output_features': 18, 'optimizer': 'Adam', 'percentile': 0.9975, 'learning_rate': 7.562830168853448e-05, 'loss': 'cross_entropy', 'activation': 'elu', 'dropout': 0.27747982301443397}. Best is trial 0 with value: 0.3076923191547394.


Train Accuracy: 0.3076923191547394, Test Accuracy: 0.375


[I 2024-11-20 22:46:15,640] Trial 1 finished with value: 0.7307692170143127 and parameters: {'output_features': 18, 'optimizer': 'SGD', 'percentile': 0.997, 'learning_rate': 0.004194098566936839, 'loss': 'cross_entropy', 'activation': 'relu', 'dropout': 0.423719617345604}. Best is trial 1 with value: 0.7307692170143127.


Train Accuracy: 0.7307692170143127, Test Accuracy: 0.5


[I 2024-11-20 22:46:17,069] Trial 2 finished with value: 0.9230769276618958 and parameters: {'output_features': 20, 'optimizer': 'RMSprop', 'percentile': 0.9981, 'learning_rate': 0.0011924239272358822, 'loss': 'cross_entropy', 'activation': 'relu', 'dropout': 0.27167443957609283}. Best is trial 2 with value: 0.9230769276618958.


...

Train Accuracy: 1.0, Test Accuracy: 0.625


[I 2024-11-20 23:22:00,171] Trial 2518 finished with value: 0.6538461446762085 and parameters: {'output_features': 18, 'optimizer': 'RMSprop', 'percentile': 0.9965, 'learning_rate': 0.00024356669115115206, 'loss': 'cross_entropy', 'activation': 'selu', 'dropout': 0.37274969555399134}. Best is trial 5 with value: 1.0.


Train Accuracy: 0.6538461446762085, Test Accuracy: 0.375


[I 2024-11-20 23:22:01,336] Trial 2519 finished with value: 0.8461538553237915 and parameters: {'output_features': 10, 'optimizer': 'RMSprop', 'percentile': 0.9981, 'learning_rate': 0.0008409369795831576, 'loss': 'cross_entropy', 'activation': 'selu', 'dropout': 0.20492690774101457}. Best is trial 5 with value: 1.0.


Train Accuracy: 0.8461538553237915, Test Accuracy: 0.25
Total computation time for all 2520 trials: 2148.0847 seconds


In [25]:
import heapq

# Assuming CT is a list of dictionaries and each dictionary has a 'test_accuracy' key
top_ten_max_results = heapq.nlargest(10, enumerate(CT), key=lambda x: x[1]['test_accuracy'])

# Print the top ten results along with their indices
for index, result in top_ten_max_results:
    print(f'Index: {index}, Test Accuracy: {result["test_accuracy"]}')

Index: 28, Test Accuracy: 0.875
Index: 88, Test Accuracy: 0.875
Index: 139, Test Accuracy: 0.875
Index: 534, Test Accuracy: 0.875
Index: 976, Test Accuracy: 0.875
Index: 1141, Test Accuracy: 0.875
Index: 1162, Test Accuracy: 0.875
Index: 1175, Test Accuracy: 0.875
Index: 1254, Test Accuracy: 0.875
Index: 1265, Test Accuracy: 0.875


In [26]:
import heapq

# Assuming CT is a list of dictionaries and each dictionary has a 'test_accuracy' key
top_ten_max_results = heapq.nlargest(20, enumerate(TT), key=lambda x: x[1]['train_accuracy'])

# Print the top ten results along with their indices
for index, result in top_ten_max_results:
    print(f'Index: {index}, Test Accuracy: {result["train_accuracy"]}')

Index: 5, Test Accuracy: 1.0
Index: 10, Test Accuracy: 1.0
Index: 11, Test Accuracy: 1.0
Index: 13, Test Accuracy: 1.0
Index: 14, Test Accuracy: 1.0
Index: 15, Test Accuracy: 1.0
Index: 16, Test Accuracy: 1.0
Index: 17, Test Accuracy: 1.0
Index: 19, Test Accuracy: 1.0
Index: 20, Test Accuracy: 1.0
Index: 21, Test Accuracy: 1.0
Index: 22, Test Accuracy: 1.0
Index: 23, Test Accuracy: 1.0
Index: 24, Test Accuracy: 1.0
Index: 26, Test Accuracy: 1.0
Index: 27, Test Accuracy: 1.0
Index: 28, Test Accuracy: 1.0
Index: 31, Test Accuracy: 1.0
Index: 32, Test Accuracy: 1.0
Index: 33, Test Accuracy: 1.0


In [27]:
train=[]
test=[]
for i in range(0,2520):
    train.append(round(TT[i]['train_accuracy'].item(),4))
    test.append(round(CT[i]['test_accuracy'].item(),4))

Final_results=pd.DataFrame()

Final_results['train']=train
Final_results['test']=test

In [28]:
pd.set_option('display.max_rows', None)
Final_results

,train,test
0,0.3077,0.375
1,0.7308,0.500
2,0.9231,0.500
3,0.6923,0.500
4,0.8462,0.500
5,1.0000,0.500
6,0.9615,0.625
7,0.6923,0.500
8,0.8077,0.375
9,0.9615,0.500


In [29]:
X_train_df

,0,1,2,3,4,5,6,7,8,9,...,758,759,760,761,762,763,764,765,766,767
0,-0.048460,0.084757,0.003497,-0.087329,0.059048,-0.209606,-0.006506,0.048831,0.017141,-0.048087,...,0.013604,0.032850,-0.101248,-0.028741,-0.000748,0.088436,0.065536,-0.118125,-0.061766,-0.032498
1,-0.065723,0.025389,-0.025804,-0.130403,0.077852,-0.120231,-0.001798,0.034838,0.010572,-0.088210,...,0.007799,0.013520,-0.135819,-0.036519,-0.043678,0.078114,0.019443,-0.130293,-0.037107,0.014137
2,-0.045842,0.040817,-0.028311,-0.119993,0.082768,-0.169268,-0.019755,0.051325,0.019362,-0.082979,...,0.033639,0.019891,-0.131964,-0.049223,-0.001145,0.100791,0.072946,-0.162404,-0.066192,0.011734
3,-0.049971,0.032408,-0.005414,-0.096241,0.066642,-0.137509,-0.046769,0.021914,0.060602,-0.076358,...,0.071187,-0.008804,-0.099127,-0.071892,-0.024793,0.082018,0.093390,-0.118121,-0.078987,0.012262
4,-0.045510,0.020915,-0.008356,-0.120176,0.046261,-0.174338,-0.013932,0.018657,0.020323,-0.080896,...,0.047783,-0.003582,-0.096276,-0.044621,-0.051367,0.058235,0.079637,-0.182815,-0.076388,0.035959
5,-0.036322,0.012869,-0.005531,-0.096341,0.042336,-0.145392,-0.009048,0.057103,0.024687,-0.081741,...,0.042723,0.028751,-0.127673,-0.079602,-0.025974,0.060759,0.061441,-0.139629,-0.045193,0.000219
6,-0.060810,0.025124,-0.007080,-0.107931,0.077159,-0.158238,-0.009238,0.045209,-0.006960,-0.061356,...,0.034085,0.035480,-0.131733,-0.027968,-0.024657,0.059341,0.037378,-0.121579,-0.073561,-0.028517
7,-0.058395,0.055415,-0.010777,-0.103418,0.084679,-0.153220,-0.018571,0.026160,0.003563,-0.045168,...,0.012902,-0.002383,-0.076134,-0.041375,-0.011632,0.093414,0.061335,-0.096002,-0.045839,0.017703
8,-0.040013,0.054013,-0.036306,-0.118734,0.021667,-0.147501,-0.008989,0.048431,0.083064,-0.052080,...,-0.003629,0.026083,-0.086470,-0.044942,-0.000232,0.104547,0.081997,-0.098181,-0.051479,-0.035658
9,-0.041498,0.088844,-0.023297,-0.116896,0.075261,-0.105749,-0.036887,0.019373,0.045887,-0.041021,...,0.005279,-0.017911,-0.042631,-0.035702,0.001695,0.110678,0.060418,-0.087090,-0.090463,0.006580


In [30]:
labels = pd.read_csv("/AnimalLabels.csv")
labels['majority_vote'] = labels.mode(axis=1, numeric_only=True).astype(int)
type(labels['majority_vote'])

pandas.core.series.Series

In [31]:
X_train_df['label']=labels['majority_vote']
y=X_train_df['label']
X=X_train_df

In [32]:
X.drop(columns=['label'],axis=1,inplace=True)
X

,0,1,2,3,4,5,6,7,8,9,...,758,759,760,761,762,763,764,765,766,767
0,-0.048460,0.084757,0.003497,-0.087329,0.059048,-0.209606,-0.006506,0.048831,0.017141,-0.048087,...,0.013604,0.032850,-0.101248,-0.028741,-0.000748,0.088436,0.065536,-0.118125,-0.061766,-0.032498
1,-0.065723,0.025389,-0.025804,-0.130403,0.077852,-0.120231,-0.001798,0.034838,0.010572,-0.088210,...,0.007799,0.013520,-0.135819,-0.036519,-0.043678,0.078114,0.019443,-0.130293,-0.037107,0.014137
2,-0.045842,0.040817,-0.028311,-0.119993,0.082768,-0.169268,-0.019755,0.051325,0.019362,-0.082979,...,0.033639,0.019891,-0.131964,-0.049223,-0.001145,0.100791,0.072946,-0.162404,-0.066192,0.011734
3,-0.049971,0.032408,-0.005414,-0.096241,0.066642,-0.137509,-0.046769,0.021914,0.060602,-0.076358,...,0.071187,-0.008804,-0.099127,-0.071892,-0.024793,0.082018,0.093390,-0.118121,-0.078987,0.012262
4,-0.045510,0.020915,-0.008356,-0.120176,0.046261,-0.174338,-0.013932,0.018657,0.020323,-0.080896,...,0.047783,-0.003582,-0.096276,-0.044621,-0.051367,0.058235,0.079637,-0.182815,-0.076388,0.035959
5,-0.036322,0.012869,-0.005531,-0.096341,0.042336,-0.145392,-0.009048,0.057103,0.024687,-0.081741,...,0.042723,0.028751,-0.127673,-0.079602,-0.025974,0.060759,0.061441,-0.139629,-0.045193,0.000219
6,-0.060810,0.025124,-0.007080,-0.107931,0.077159,-0.158238,-0.009238,0.045209,-0.006960,-0.061356,...,0.034085,0.035480,-0.131733,-0.027968,-0.024657,0.059341,0.037378,-0.121579,-0.073561,-0.028517
7,-0.058395,0.055415,-0.010777,-0.103418,0.084679,-0.153220,-0.018571,0.026160,0.003563,-0.045168,...,0.012902,-0.002383,-0.076134,-0.041375,-0.011632,0.093414,0.061335,-0.096002,-0.045839,0.017703
8,-0.040013,0.054013,-0.036306,-0.118734,0.021667,-0.147501,-0.008989,0.048431,0.083064,-0.052080,...,-0.003629,0.026083,-0.086470,-0.044942,-0.000232,0.104547,0.081997,-0.098181,-0.051479,-0.035658
9,-0.041498,0.088844,-0.023297,-0.116896,0.075261,-0.105749,-0.036887,0.019373,0.045887,-0.041021,...,0.005279,-0.017911,-0.042631,-0.035702,0.001695,0.110678,0.060418,-0.087090,-0.090463,0.006580


In [33]:
indices_to_change = [4, 105, 84, 27, 98, 88, 18, 65, 9, 2, 5, 49, 99, 69, 86, 67, 7, 28, 78, 70, 18, 74]

# Create a boolean mask to identify indices for the test set
test_mask = np.zeros(len(X), dtype=bool)
test_mask[indices_to_change] = True

# Split the dataset based on the boolean mask
X_train = X[~test_mask]
X_test = X[test_mask]
y_train = y[~test_mask]
y_test = y[test_mask]

In [34]:
param_grid = {'penalty':['l1','l2',None],'C': [0.1,2,2.5,3],
                   'solver':['liblinear','saga'],'max_iter':[500]}
LR=LogisticRegression(random_state=0)
grid_search_LR = GridSearchCV(estimator=LR,
                           param_grid=param_grid,
                           cv=3)

grid_search_LR.fit(X_train, y_train)
print("Best parameters found: ", grid_search_LR.best_params_)
print("Best Score: {}".format(grid_search_LR.best_score_))

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_sag.py:349: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_sag.py:349: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:1208: UserWarning: Setting penalty=None will ignore the C and l1_ratio parameters
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_sag.py:349: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:1208: UserWarning: Setting penalty=None will ignore the C and l1_ratio parameters
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_sag.py:349: ConvergenceWarning: The max_iter was reached which mean

Best parameters found:  {'C': 0.1, 'max_iter': 500, 'penalty': None, 'solver': 'saga'}
Best Score: 0.6588669950738916


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_sag.py:349: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


In [35]:
param_grid = {'penalty':['elasticnet',None],'C': [0.1,2,2.5,3],
                   'solver':['saga'],'max_iter':[500],'l1_ratio':[0.5,0.2,0.7]}
LR=LogisticRegression(random_state=0)
grid_search_LR = GridSearchCV(estimator=LR,
                           param_grid=param_grid,
                           cv=2)
grid_search_LR.fit(X_train, y_train)
print("Best parameters found: ", grid_search_LR.best_params_)
print("Best Score: {}".format(grid_search_LR.best_score_))

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:1197: UserWarning: l1_ratio parameter is only used when penalty is 'elasticnet'. Got (penalty=None)
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:1208: UserWarning: Setting penalty=None will ignore the C and l1_ratio parameters
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_sag.py:349: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:1197: UserWarning: l1_ratio parameter is only used when penalty is 'elasticnet'. Got (penalty=None)
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:1208: UserWarning: Setting penalty=None will ignore the C and l1_ratio parameters
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_sag.py:349: ConvergenceWarning: The

Best parameters found:  {'C': 0.1, 'l1_ratio': 0.5, 'max_iter': 500, 'penalty': None, 'solver': 'saga'}
Best Score: 0.670265780730897


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_sag.py:349: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


In [36]:
param_grid = {'penalty':['l2',None],'C': [0.1,2,2.5,3],
                   'solver':['lbfgs','newton-cg','sag'],'max_iter':[500]}
LR=LogisticRegression(random_state=0)
grid_search_LR = GridSearchCV(estimator=LR,
                           param_grid=param_grid,
                           cv=3
                          )

grid_search_LR.fit(X_train, y_train)
print("Best parameters found: ", grid_search_LR.best_params_)
print("Best Score: {}".format(grid_search_LR.best_score_))

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_sag.py:349: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_sag.py:349: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:1208: UserWarning: Setting penalty=None will ignore the C and l1_ratio parameters
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:1208: UserWarning: Setting penalty=None will ignore the C and l1_ratio parameters
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:1208: UserWarning: Setting penalty=None will ignore the C and l1_ratio parameters
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:1208: UserWarning: Setting penalty=None will ignore the C

Best parameters found:  {'C': 0.1, 'max_iter': 500, 'penalty': 'l2', 'solver': 'lbfgs'}
Best Score: 0.6469622331691297


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_sag.py:349: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


In [37]:
LR=LogisticRegression(C=2.5,penalty='elasticnet',solver='saga',max_iter=1000,l1_ratio= 0.2)
LR.fit(X_train,y_train)

LogisticRegression(C=2.5, l1_ratio=0.2, max_iter=1000, penalty='elasticnet',
                   solver='saga')

In [38]:
LR_pred = LR.predict(X_train)
acc_train=accuracy_score(y_train,LR_pred)
print('train_accuracy',acc_train)
LR_pred = LR.predict(X_test)
acc=accuracy_score(y_test,LR_pred)
print('test_accuracy',acc)

train_accuracy 0.6470588235294118
test_accuracy 0.6666666666666666


In [39]:
X_train_values = X_train.values
X_test_values = X_test.values
#X_valid_values = X_valid.values

# Reshape input data to 3D [samples, timesteps, features] so it'll fit the LSTM layer
# Use a timestep of 1.
X_train_reshaped = np.reshape(X_train_values, (X_train_values.shape[0], 1, X_train_values.shape[1]))
X_test_reshaped = np.reshape(X_test_values, (X_test_values.shape[0], 1, X_test_values.shape[1]))
#X_valid_reshaped = np.reshape(X_valid_values, (X_valid_values.shape[0], 1, X_valid_values.shape[1]))

# LSTM model
lstm = Sequential()
lstm.add(LSTM(100,return_sequences=True,activation='selu',input_shape=(X_train_reshaped.shape[1], X_train_reshaped.shape[2])))
lstm.add(Dropout(0.7))

lstm.add(LSTM(50,activation='selu',return_sequences=False,kernel_regularizer=l2(0.01)))
lstm.add(Dropout(0.7))
lstm.add(Dense(1,activation='sigmoid'))  # Prediction of the next closing value


# Compile the model
lstm.compile(optimizer='adam', loss='binary_crossentropy')

# Train the model
lstm.fit(X_train_reshaped, y_train, epochs=1000, batch_size=32, verbose=1)

# Predicting and inverse transforming the predictions
y_pred_train_lstm = lstm.predict(X_train_reshaped)
y_pred_test_lstm = lstm.predict(X_test_reshaped)

#y_pred = scaler.inverse_transform(y_pred)

/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/1000
3/3 ━━━━━━━━━━━━━━━━━━━━ 5s 13ms/step - loss: 2.0424
Epoch 2/1000
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 1.9783 
Epoch 3/1000
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 1.8649
Epoch 4/1000
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 1.8079 
Epoch 5/1000
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 1.7574
Epoch 6/1000
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - loss: 1.6799
Epoch 7/1000
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 1.5948 
Epoch 8/1000
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 1.5715
Epoch 9/1000
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - loss: 1.5093
Epoch 10/1000
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - loss: 1.4421 
Epoch 11/1000
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 1.3887
Epoch 12/1000
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 1.3237
Epoch 13/1000
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 1.3278 
Epoch 14/1000
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 1.3117
Epoch 15/1000
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 1.2574
Epo

In [40]:
y_pred_train_lstm = np.squeeze(y_pred_train_lstm)
y_pred_test_lstm = np.squeeze(y_pred_test_lstm)
y_pred_train_lstm = (y_pred_train_lstm > 0.5).astype(int)
acc_train=accuracy_score(y_train,y_pred_train_lstm)
print('train accuracy',acc_train)
y_pred_test_lstm = (y_pred_test_lstm > 0.5).astype(int)
acc_test=accuracy_score(y_test,y_pred_test_lstm)
print('test accuracy',acc_test)

train accuracy 1.0
test accuracy 0.6190476190476191


In [41]:
bilstm = Sequential()
bilstm.add(Bidirectional(LSTM(100, return_sequences=True,kernel_regularizer=l2(0.01)), input_shape=(X_train_reshaped.shape[1], X_train_reshaped.shape[2])))
#bilstm.add(Dropout(0.7))
bilstm.add(Bidirectional(LSTM(50, return_sequences=False,kernel_regularizer=l2(0.01))))
#bilstm.add(Dropout(0.7))
bilstm.add(Dense(1, activation='sigmoid',kernel_regularizer=l2(0.01)))  # Binary classification output

# Compile the model
bilstm.compile(optimizer='adam', loss='binary_crossentropy')

# Train the model
bilstm.fit(X_train_reshaped, y_train, epochs=500, batch_size=32, verbose=1)

# Predicting
y_pred_train_bilstm = bilstm.predict(X_train_reshaped)
y_pred_test_bilstm = bilstm.predict(X_test_reshaped)

Epoch 1/500


/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/bidirectional.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


3/3 ━━━━━━━━━━━━━━━━━━━━ 6s 18ms/step - loss: 14.9206
Epoch 2/500
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - loss: 13.3887 
Epoch 3/500
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - loss: 11.9818 
Epoch 4/500
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - loss: 10.6961
Epoch 5/500
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step - loss: 9.5409
Epoch 6/500
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - loss: 8.5089
Epoch 7/500
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - loss: 7.5822 
Epoch 8/500
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - loss: 6.7269
Epoch 9/500
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - loss: 5.9781
Epoch 10/500
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - loss: 5.3193
Epoch 11/500
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - loss: 4.7373
Epoch 12/500
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - loss: 4.2337 
Epoch 13/500
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - loss: 3.7796
Epoch 14/500
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - loss: 3.3775
Epoch 15/500
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step - loss: 3.0318
Epoch 16/500
3/3 ━━━━━━

In [42]:
y_pred_train_bilstm = np.squeeze(y_pred_train_bilstm)
y_pred_test_bilstm = np.squeeze(y_pred_test_bilstm)

In [43]:
y_pred_train_bilstm = (y_pred_train_bilstm > 0.5).astype(int)
acc_train=accuracy_score(y_train,y_pred_train_bilstm)
print('train accuracy',acc_train)
y_pred_test_bilstm = (y_pred_test_bilstm > 0.5).astype(int)
acc_test=accuracy_score(y_test,y_pred_test_bilstm)
print('test accuracy',acc_test)

train accuracy 0.6470588235294118
test accuracy 0.6666666666666666
